In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment


def getData(GameId):
    url = f'https://www.pro-football-reference.com/boxscores/{GameId}.htm'
    req = requests.get(url).text
    bs = BeautifulSoup(req).find(id='all_pbp')
    comments = bs.find_all(string=lambda text: isinstance(text, Comment))
    table = [c for c in comments if len(c) > 10000][0]
    df = pd.read_html(table)[0]


    df_filtered = df[df['Quarter'].isin(["1","2","3","4"])]
    df_filtered.columns
    df_filtered["TeamH"] = df_filtered.columns[6]
    df_filtered["TeamA"] = df_filtered.columns[5]
    df_filtered.rename(columns={df_filtered.columns[5]: 'Away', df_filtered.columns[6]: 'Home'}, inplace=True)
    
    return df_filtered

#phi = getData("202411140phi")
#chi = getData("202411170chi")
det = getData("202411170det")
#mia = getData("202411170mia")
#nor = getData("202411170nor")
nwe = getData("202411170nwe")
nyj = getData("202411170nyj")

df_combined = pd.concat([nyj,det,nwe], ignore_index=True)


C:\Users\scwvi\AppData\Local\Temp\ipykernel_21548\161982774.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]
C:\Users\scwvi\AppData\Local\Temp\ipykernel_21548\161982774.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["TeamH"] = df_filtered.columns[6]
C:\Users\scwvi\AppData\Local\Temp\ipykernel_21548\161982774.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [1]:
#df_combined

In [3]:
# Install openai python library
! pip install openai
import info

In [ ]:
import os
import info
os.environ["OPENAI_API_KEY"]= info.key
os.environ["OPENAI_ORG"] = info.org
os.environ["OPENAI_PROJ"] = info.proj

In [5]:
# Read authentication keys from environmental variables


_open_ai_tkn = os.environ.get("OPENAI_API_KEY")
_project_tkn = os.environ.get("OPENAI_PROJ")
_organisation_tkn = os.environ.get("OPENAI_ORG")
if not _open_ai_tkn:
    raise ValueError("API key is missing. Ensure OPENAI_API_KEY is set in the environment variables.")

In [6]:
# create client endpoint for accessing remote LLM
from openai import OpenAI

client = OpenAI(
  organization=_organisation_tkn,
  project=_project_tkn,
  api_key=_open_ai_tkn
)

In [7]:
# Making a request
promptResults = []
for i in range(0,len(df_combined)):
#for i in range(0,10):
    awayS = df_combined.loc[i, 'Away']
    homeS = df_combined.loc[i, 'Home']
    awayT = df_combined.loc[i, 'TeamA']
    homeT = df_combined.loc[i, 'TeamH']
    time = df_combined.loc[i, 'Time']
    down = df_combined.loc[i, 'Down']
    togo = df_combined.loc[i, 'ToGo']
    play = df_combined.loc[i, 'Detail']
    quarter = df_combined.loc[i, 'Quarter']
    if down == None:
        down = ""
    elif down == 1:
        down = "1st"
    elif down == 2:
        down = "2nd"
    elif down == 3:
        down = "3rd"
    elif down == 4:
        down = "4th"    
    
    location = df_combined.loc[i, 'Location']

    response = client.chat.completions.create(
        messages=[{
            "role": "user",
            "content": f"Give realistic, short and concise commentary for this play. Use no outside context, and just this information. Make sure all information in commentary matches the information given. Respond only with the commentary. Quarter {quarter}, {time} time left, {down} down, {togo} yards to go at {location}. Score: {homeT}:{homeS}, {awayT}:{awayS}. Play: {play}"
        }],
        model="gpt-4o-mini",
        max_completion_tokens=1000
    )
    promptResults.append((response.choices[0].message.content,f"Quarter:{quarter}, Time:{time}, Down:{down}, Yards to go:{togo}, At:{location}. Score: {homeT}:{homeS}, {awayT}:{awayS}. Play:{play}"))

In [2]:
#for i in range(0,10):
    #print(promptResults[i])
    #print(promptResults)

In [12]:
#print(len(promptResults))
df_toview = pd.DataFrame(promptResults,columns=["Playcall", "Information"])
df_toview

df_toview.to_csv("AIcommentary.csv")
